# Genetic Matching for Robustness and Model Independence

__Tasks__

[ ] Get the nsw dataset

[ ] # SECOND OPTION: Maximize gamma associated with losing statistical significance instead of the max pvalue of gamma = 1.5

[ ] who has cited Athey and Imbens?

## Prelude

Let's get a taste of the libraries we will be using.

In [1]:
# Load imports
library(rbounds)
library(rgenoud)

# Load data
data(lalonde)
attach(lalonde)

Loading required package: Matching
Loading required package: MASS
## 
##  Matching (Version 4.9-3, Build Date: 2018-05-03)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

##  rgenoud (Version 5.8-2.0, Build Date: 2018-04-03)
##  See http://sekhon.berkeley.edu/rgenoud for additional documentation.
##  Please cite software as:
##   Walter Mebane, Jr. and Jasjeet S. Sekhon. 2011.
##   ``Genetic Optimization Using Derivatives: The rgenoud package for R.''
##   Journal of Statistical Software, 42(11): 1-26. 
##



In [2]:
# Create data objects
Y <- re78
Tr <- treat
X <- cbind(age, educ, black, hisp, married, nodegr, re74, re75, u74, u75)
BalanceMat <- cbind(age, I(age^2), educ, I(educ^2), black,
                    hisp, married, nodegr, re74 , I(re74^2), re75, I(re75^2),
                    u74, u75, I(re74*re75), I(age*nodegr), I(educ*re74), I(educ*75))

In [3]:
set.seed(12345)
# Genetic matching 
gen1 <- GenMatch(Tr=Tr, X=X, BalanceMat=BalanceMat, pop.size=50,
                 data.type.int=FALSE, print=1, replace=FALSE)



Mon Feb 17 14:59:32 2020
Domains:
 0.000000e+00   <=  X1   <=    1.000000e+03 
 0.000000e+00   <=  X2   <=    1.000000e+03 
 0.000000e+00   <=  X3   <=    1.000000e+03 
 0.000000e+00   <=  X4   <=    1.000000e+03 
 0.000000e+00   <=  X5   <=    1.000000e+03 
 0.000000e+00   <=  X6   <=    1.000000e+03 
 0.000000e+00   <=  X7   <=    1.000000e+03 
 0.000000e+00   <=  X8   <=    1.000000e+03 
 0.000000e+00   <=  X9   <=    1.000000e+03 
 0.000000e+00   <=  X10  <=    1.000000e+03 

Data Type: Floating Point
Operators (code number, name, population) 
	(1) Cloning........................... 	7
	(2) Uniform Mutation.................. 	6
	(3) Boundary Mutation................. 	6
	(4) Non-Uniform Mutation.............. 	6
	(5) Polytope Crossover................ 	6
	(6) Simple Crossover.................. 	6
	(7) Whole Non-Uniform Mutation........ 	6
	(8) Heuristic Crossover............... 	6
	(9) Local-Minimum Crossover........... 	0

SOFT Maximum Number of Generations: 100
Maximum Nonchang

In [4]:
# Matching
mgen1 <- Match(Y=Y, Tr=Tr, X=X, Weight.matrix=gen1, replace=FALSE)
summary(mgen1)


Estimate...  1789.4 
SE.........  705.77 
T-stat.....  2.5354 
p.val......  0.011233 

Original number of observations..............  445 
Original number of treated obs...............  185 
Matched number of observations...............  185 
Matched number of observations  (unweighted).  185 



In [6]:
# Balance
mbgen1 <- MatchBalance(Tr ~ age + I(age^2)+ educ + I(educ^2) + black +
                       hisp + married + nodegr + re74 + I(re74^2) + re75 + I(re75^2) +
                       u74 + u75 + I(re74*re75) + I(age*nodegr) + I(educ*re74) + I(educ*75),
                       data=lalonde, match.out = mgen1, print.level=0)

[1] 0.0381402
[1] 0.0381402


In [7]:
# Sensitivity Analysis
p <- psens(mgen1, Gamma=1.5, GammaInc=.1)
p
1 - p$bounds$`Upper bound`
hlsens(mgen1, Gamma=1.5, GammaInc=.1)

Gamma,Lower bound,Upper bound
1.0,0.0103,0.0103
1.1,0.0021,0.0376
1.2,0.0004,0.0979
1.3,0.0001,0.1981
1.4,0.0000,0.3309
1.5,0.0000,0.4778


[1] 0.9897 0.9624 0.9021 0.8019 0.6691 0.5222

Gamma,Lower bound,Upper bound
1.0,1584.300,1584.3
1.1,970.200,1717.9
1.2,622.300,2016.4
1.3,332.300,2276.1
1.4,88.195,2575.9
1.5,-91.005,2850.4


## Match vs. GenMatch Investigation 

In [8]:
# Are matches from the genetic matching object the same and the ones from the match object?
m = sum(gen1$matches[, 2] != mgen1$index.control)
n = nrow(gen1$matches)
print(sprintf("%d / %d are different matches", m, n))

[1] "39 / 185 are different matches"


In [9]:
# Is it because of the stochastic behavior of ties = FALSE?

gen2 <- GenMatch(Tr=Tr, X=X, BalanceMat=BalanceMat, pop.size=50,
                 data.type.int=FALSE, print=0)

mgen2 <- Match(Y=Y, Tr=Tr, X=X, Weight.matrix=gen2)

mbgen2 <- MatchBalance(Tr ~ age + I(age^2)+ educ + I(educ^2) + black +
                       hisp + married + nodegr + re74 + I(re74^2) + re75 + I(re75^2) +
                       u74 + u75 + I(re74*re75) + I(age*nodegr) + I(educ*re74) + I(educ*75),
                       data=lalonde, match.out = mgen2, print=0)

stopifnot(gen1$value[1] == mbgen1$AMsmallest.p.value)

m = sum(gen2$matches[, 2] != mgen2$index.control)
n = nrow(gen2$matches)
print(sprintf("%d / %d are different matches", m, n))

print("yes :)")

[1] "0 / 272 are different matches"
[1] "yes :)"


### Thoughts

There is an issue moving forward. Since the fit function is computed over the matches generated by `GenMatch`, with `ties=FALSE` we cannot guarantee that the loss of the `Match` output will be the same as the `GenMatch` output. The cause seems to be that an equal distance is the matching space (i.e., a tie among controls) does not entail that the ties controls result in the same level of robustness. In other words, a tie in the "matching space" is not equivalent to a tie in the "robustness space".

Note that the only reason we do not want to allow ties is because the `psens` function does not account for it in its calculation of Rosenbaum's $\Gamma$. A possible way to resolve the lack of stability in `GenMatch + psens` is to allow ties, but break them deterministically instead of stochastically inside the fitness function, and then do the same with the output of `Match`. For example, since the tied controls are ordered numerically, one could pick the first control only.

In [10]:
# Example
first.tie.trt.idx <- which(gen2$matches[,3] < 1)[1]
first.tie.rows.idx <- which(gen2$matches[,1] == first.tie.trt.idx)
gen2$matches[first.tie.rows.idx, ]
sprintf("Instead of using %d controls, we could deterministically pick the first one", 
        length(first.tie.rows.idx))

6,188,0.5
6,244,0.5


[1] "Instead of using 2 controls, we could deterministically pick the first one"

## Robustness Optimization with `rbounds`

In [11]:
# Sensitivity optim function
robust.fitfunc <- function(matches, BM) {

  # my.fitfunc requires that the last column of the BM matrix contain the outcomes
  # hence, it is VERY important not to run standard pval balance genetic matching
  # with the same BM!
  
  # NOTE: psens does NOT acount for 1:many matching!
  index.treated <- matches[,1]
  index.control <- matches[,2]
  
  # Get treated and control unit outcomes
  trt <-  BM[, ncol(BM)][index.treated]
  ctrl <- BM[, ncol(BM)][index.control]
  
  # Compute sensitivity
  psens.out <- psens(trt, ctrl, Gamma=1.5, GammaInc=.1)
  
  # We want to minimize the largest p-value. Thus,
  pvals <- psens.out$bounds$`Upper bound`
  
  return(sort(pvals, decreasing=TRUE))
}

### Fitness Function Demonstration

In [12]:
# Demonstration ties=FALSE
BM <- cbind(BalanceMat, re78)
df <- data.frame(trt = mgen1$index.treated, ctrl = mgen1$index.control)
robust.fitfunc(df, BM)           # With matches from Match()
robust.fitfunc(gen1$matches, BM) # With matches from GenMatch()

[1] 0.4778 0.3309 0.1981 0.0979 0.0376 0.0103

[1] 0.3194 0.1986 0.1052 0.0452 0.0148 0.0034

In [13]:
# with ties=TRUE the results are the same, but the pvals are probably incorrect
BM <- cbind(BalanceMat, re78)
df <- data.frame(trt = mgen2$index.treated, ctrl = mgen2$index.control)
robust.fitfunc(df, BM)           # With matches from Match()
robust.fitfunc(gen2$matches, BM) # With matches from GenMatch()

[1] 0.7011 0.5261 0.3333 0.1667 0.0603 0.0140

[1] 0.7011 0.5261 0.3333 0.1667 0.0603 0.0140

### Genetic Matching 
Before resolving the issue of ties, let us check whether genetic matching works in this simple case.

In [14]:
# Genetic Optimization
genout3 <- GenMatch(Tr=Tr, X=X, BalanceMat=Y, pop.size=500,
                   print=1, ties=FALSE, wait.generations = 20, fit.func = robust.fitfunc)

mout <- Match(Y=Y, Tr=treat, X=X, ties=FALSE, Weight.matrix=genout3)
summary(mout)



Mon Feb 17 15:00:17 2020
Domains:
 0.000000e+00   <=  X1   <=    1.000000e+03 
 0.000000e+00   <=  X2   <=    1.000000e+03 
 0.000000e+00   <=  X3   <=    1.000000e+03 
 0.000000e+00   <=  X4   <=    1.000000e+03 
 0.000000e+00   <=  X5   <=    1.000000e+03 
 0.000000e+00   <=  X6   <=    1.000000e+03 
 0.000000e+00   <=  X7   <=    1.000000e+03 
 0.000000e+00   <=  X8   <=    1.000000e+03 
 0.000000e+00   <=  X9   <=    1.000000e+03 
 0.000000e+00   <=  X10  <=    1.000000e+03 

Data Type: Floating Point
Operators (code number, name, population) 
	(1) Cloning........................... 	65
	(2) Uniform Mutation.................. 	62
	(3) Boundary Mutation................. 	62
	(4) Non-Uniform Mutation.............. 	62
	(5) Polytope Crossover................ 	62
	(6) Simple Crossover.................. 	62
	(7) Whole Non-Uniform Mutation........ 	62
	(8) Heuristic Crossover............... 	62
	(9) Local-Minimum Crossover........... 	0

SOFT Maximum Number of Generations: 100
Maximum 

In [15]:
# Notice that the pvalues from inside genetic matching 
# and from the match object are very different
rbind(sort(genout3$value),
      psens(mout, Gamma=1.5, GammaInc=.1)$bounds[, 3])

0e+00,0.0001,0.0006,0.0027,0.0085,0.0217
2e-04,0.0015,0.0066,0.0205,0.0502,0.1016


## Robustness Optimization with `sensitivitymv` 

In [16]:
library(sensitivitymult)

In [17]:
create_sets <- function(matches, BM) {
    
    # Get number of units and the number of rows in the matches dataframe 
    n_rows <- nrow(BM)
    row_max <- nrow(matches)

    # Get number of sets (n = number of treated units)
    n_sets <- length(unique(matches[,1]))

    # Create vector to store matched sets
    matched_sets <- rep(NA, n_rows)

    # Row pointer that will traverse the matches object
    row_pointer <- 1

    # Get index.treat and index.control
    index.treat <- matches[,1]
    index.control <- matches[,2]

    for (set.idx in 1:n_sets) {

        # Get the treated unit index of the current set
        current.t.idx <- index.treat[row_pointer]
        first.c.idx <- index.control[row_pointer]

        # Store set index at position corresponding to treated and first control units
        matched_sets[current.t.idx] <- set.idx
        matched_sets[first.c.idx] <- set.idx

        # Are there more controls in this set? If so, find them.
        while (row_pointer < row_max && index.treat[row_pointer + 1] == current.t.idx) {

            row_pointer <- row_pointer + 1             # Increase row pointer
            next.c.idx <- index.control[row_pointer]   # Get index of next control in the set
            matched_sets[next.c.idx] <- set.idx        # Store set index at corresponding position
        }

        # After finding all controls in this set, increase row pointer 
        # before going to the next set
        row_pointer <- row_pointer + 1  

    }
    
    df <- data.frame(BM, matched_sets)[!is.na(matched_sets),]
    df <- df[order(df$matched_sets),]

    return(matched_sets)
}

In [18]:
data_prep <- create_sets(gen2$matches, data.frame(re78, treat))

In [19]:
senm(y=data_prep$re78, z=data_prep$treat, mset=data_prep$matched_sets, 
     gamma=1, inner=0, trim=Inf, lambda = 1/2, TonT=TRUE)

ERROR: Error in is.vector(y) & is.vector(z) & is.vector(mset): $ operator is invalid for atomic vectors


It breaks with replace...

In [30]:
# Okay I created this piece of shit function so that I could have data from matches
# ready to be input by the format required by senmv. Hopefully it works now?
# Note that I went directly for senmv because if I tried to get it in form
# for senm in sensitivitymult it would be a big waste of time since it would use it
# to reconstruct the matrix any way (apparently at least). so here we are.
# Hopefully the next step would be to check whether I'm successful in passing this matrix
# to senmv to get the p values. If so, it should hopefully be easy to put in the optim func
# and be done with it yay

make_ymat<-function(y, matches){
  
  treated <- matches[, 1]
  controls <- matches[, 2]
  trt_table <- table(treated)
  n_sets <- length(trt_table)
  max_ctrls <- max(trt_table)
  
  # Smallest table necessary is number of sets vs. size of largest set
  y_mat <- matrix(NA, n_sets, max_ctrls + 1)
  
  m <- 0 # Auxiliary indexer
  
  # For each set
  for (i in 1:n_sets){
    
    y_mat[i, 1] <- y[as.numeric(names(trt_table)[i])] # Get treated outcome
    y_mat[i, 2:(1+trt_table[i])] <- y[(m+1):(m+trt_table[i])]
    m<-m+trt_table[i]
  }
  y_mat
}

ymat <- make_ymat(re78, matches)

ERROR: Error in make_ymat(re78, matches): object 'matches' not found


In [ ]:
senm(y=data_prep$re78, z=data_prep$treat, mset=data_prep$matched_sets, 
     gamma=1, inner=0, trim=Inf, lambda = 1/2, TonT=TRUE)